# Notebook Imports

In [129]:
%pip install ultralytics
%pip install cv2
%pip install tensorflow
%pip install haversine

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [130]:
from ultralytics import YOLO, solutions
from ultralytics.solutions import speed_estimation
from collections import defaultdict

import haversine as hs   
from haversine import Unit

import torch
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchvision.models.detection import ssd300_vgg16, fasterrcnn_resnet50_fpn_v2

import numpy as np
import cv2
import math
import time
import json
import os

## Video Details

In [131]:
IMAGE_PATH = "seattle13.jpg"
VIDEO_PATH = "highway.mp4"

SPEED_LIMIT = 70

## COCO Names

In [132]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
    'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
    'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
    'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
    'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'TV', 'laptop',
    'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
    'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    'teddy bear', 'hair drier', 'toothbrush'
]

## Loading the Data from JSON file

In [133]:
with open('points.json') as file:
    file_contents = json.load(file)

## Main Functions of Speed Estimation

### Euclidean Distance

In [134]:
def euclidean_distance(point_a: tuple, point_b: tuple) -> float:

    """
    Calculate the Euclidean distance between two points.

    Parameters:
    - point_a (tuple): (x, y) coordinates of the first point.
    - point_b (tuple): (x, y) coordinates of the second point.

    Returns:
    float: Euclidean distance between the two points.
    """

    x_a, y_a = point_a
    x_b, y_b = point_b
    distance = round(math.sqrt((x_b - x_a) ** 2 + (y_b - y_a) ** 2), 3)
    return distance

### Scaling Factor

In [135]:
def scaling_factor(dict_file: dict) -> float :

    """
    Calculate the scaling factor between real-world distances and pixel distances.

    This function computes a scaling factor that converts distances measured in pixels
    to distances measured in real-world units (meters). The input dictionary is expected 
    to contain coordinates in both real-world (latitude and longitude) and pixel formats.

    Parameters:
        dict_file (dict): A dictionary with the following keys:
            - 'pts': A list of two tuples, each containing latitude and longitude coordinates.
            - '2dpts': A list of two tuples, each containing x and y pixel coordinates.

    Returns:
        float: The scaling factor representing the ratio of the real-world distance 
               (in meters) to the pixel distance.

    Example:
        >>> dict_file = {
                'pts': [(37.7749, -122.4194), (34.0522, -118.2437)],
                '2dpts': [(100, 200), (300, 400)]
            }
        >>> scaling_factor(dict_file)
        0.45
    """

    coordinate_1 = tuple(dict_file['pts'][0])
    coordinate_2 = tuple(dict_file['pts'][1])
    
    REAL_WORLD_DISTANCE = round(hs.haversine(coordinate_1, coordinate_2, unit=Unit.METERS))
    X = tuple(dict_file['2dpts'][0])
    Y = tuple(dict_file['2dpts'][1])

    PIXEL_DISTANCE = round(euclidean_distance(X, Y))
    SCALING_FACTOR = round(REAL_WORLD_DISTANCE /  PIXEL_DISTANCE, 2)
    return SCALING_FACTOR

In [136]:
SCALING_FACTOR = scaling_factor(dict_file=file_contents)
SCALING_FACTOR

0.33

### Calculate Speed

In [153]:
def calculate_speed(current_center:tuple, previous_center:tuple, scaling_factor:float, 
                    time_interval:float, frame, points:tuple):

    """
    Calculate and display the speed of a moving object in the video frame.

    Parameters:
    - current_center (tuple): (x, y) coordinates of the current center of the car.
    - previous_center (tuple): (x, y) coordinates of the previous center of the car.
    - scaling_factor (float): Scaling factor to convert pixel distance to real-world distance.
    - time_interval (float): Time interval between frames.
    - frame: Current video frame.
    - points (tuple): (x, y) coordinates for placing the speed text in the frame.
    """

    distance_pixels = euclidean_distance(current_center, previous_center)
    distance_meters = distance_pixels * scaling_factor
    speed = round(distance_meters / time_interval * 3.6)
        
    speed_text = f"Speed: {speed} km/h"
    color = (0, 0, 250) if speed > SPEED_LIMIT else (0, 0, 0)

    cv2.putText(frame, speed_text, points, cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)

## YOLOv8

In [138]:
output = cv2.VideoWriter("yolov8.mp4", cv2.VideoWriter_fourcc(*'DIVX'), 30, (1920, 1080))
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("Error opening video stream or file")
    exit()

num_frames = 0
start_time = time.time()

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    num_frames += 1

    # YOLOv8 object tracking
    predictions_yolo = model.track(frame, persist=True, classes=2, show_labels=False, show_conf=False,
                            tracker="bytetrack.yaml")

    # Visualize the results on the frame
    annotated_frame = predictions_yolo[0].plot()

    end_time = time.time()
    elapsed_time = end_time - start_time
    YOLO_FPS = num_frames / elapsed_time

    cv2.imshow("YOLOv8 Tracking", annotated_frame)
    output.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
output.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 109.0ms
Speed: 4.0ms preprocess, 109.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 174.5ms
Speed: 4.0ms preprocess, 174.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 104.2ms
Speed: 5.5ms preprocess, 104.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 101.5ms
Speed: 2.0ms preprocess, 101.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 114.0ms
Speed: 2.3ms preprocess, 114.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 104.0ms
Speed: 5.4ms preprocess, 104.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 131.0ms
Speed: 3.2ms preprocess, 131.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 133.7ms
Speed: 2.5ms preprocess, 133.7ms inference, 2.0ms postprocess per image at shape (1, 3, 38

## Faster R-CNN 

In [139]:
# Load a pre-trained Faster R-CNN model
model_02 = fasterrcnn_resnet50_fpn_v2(pretrained=True, progress=False)
model_02.eval()  # Set the model to inference mode

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [140]:
# Function to load frame
def load_frame(frame):
    frame_tensor = F.to_tensor(frame)
    return frame_tensor

In [141]:
# Function to visualize predictions on a frame using OpenCV
def visualize_predictions(frame, prediction, threshold=0.6):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = F.to_tensor(frame_rgb)
    frame_np = (frame_tensor.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    frame_bgr = cv2.cvtColor(frame_np, cv2.COLOR_RGB2BGR)

    boxes = prediction[0]['boxes'].cpu().numpy()
    scores = prediction[0]['scores'].cpu().numpy()
    labels = prediction[0]['labels'].cpu().numpy()

    car_boxes = []
    
    for box, score, label in zip(boxes, scores, labels):
        if score > threshold and label < len(COCO_INSTANCE_CATEGORY_NAMES):
            if COCO_INSTANCE_CATEGORY_NAMES[label] == 'car':
                try:
                    x_min, y_min, x_max, y_max = map(int, box)
                    cv2.rectangle(frame_bgr, (x_min, y_min), (x_max, y_max), (0, 165, 255), 2) 
                    cv2.putText(frame_bgr, f'{COCO_INSTANCE_CATEGORY_NAMES[label]}: {score:.2f}', 
                                (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 2)
                    car_boxes.append(box)
                except Exception as e:
                    print(f"Error drawing rectangle or text: {e}")
    
    return frame_bgr, car_boxes

In [142]:
# Open the video file
cap = cv2.VideoCapture(VIDEO_PATH)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video writer to save the annotated video
output = cv2.VideoWriter('faster-rcnn.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
                                                                                   int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

num_frames = 0
start_time = time.time()

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_tensor = load_frame(frame)

    # Make predictions
    with torch.no_grad():
        prediction_faster_rcnn = model_02([frame_tensor])

    # Visualize predictions
    annotated_frame, car_boxes_f = visualize_predictions(frame, prediction_faster_rcnn)

    end_time = time.time()
    elapsed_time = end_time - start_time
    FASTER_RCNN_FPS = num_frames / elapsed_time

    # Display the annotated frame
    cv2.imshow('Predictions', annotated_frame)

    # Write the annotated frame to the output video
    output.write(annotated_frame)

    # Press 'q' to exit the video early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects
cap.release()
output.release()
cv2.destroyAllWindows()

## SSD

In [143]:
model_03 = ssd300_vgg16(pretrained=True)
model_03.eval()

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=

In [144]:
# Open the video file
cap = cv2.VideoCapture(VIDEO_PATH)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video writer to save the annotated video
output = cv2.VideoWriter('ssd.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
                                                                          int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

num_frames = 0
start_time = time.time()

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_tensor = load_frame(frame)

    # Make predictions
    with torch.no_grad():
        prediction_ssd = model_03([frame_tensor])

    # Visualize predictions
    annotated_frame, car_boxes_s = visualize_predictions(frame, prediction_ssd, threshold=0.5)

    end_time = time.time()
    elapsed_time = end_time - start_time
    SSD_FPS = num_frames / elapsed_time

    # Display the annotated frame
    cv2.imshow('Predictions', annotated_frame)

    # Write the annotated frame to the output video
    output.write(annotated_frame)

    # Press 'q' to exit the video early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects
cap.release()
output.release()
cv2.destroyAllWindows()

## Comparison

### Model Size

* **YOLOv8:** 12 MB
* **Faster RCNN:** 167 MB
* **SSD:** 136 MB

In [145]:
def get_model_size(model, model_path):
    torch.save(model.state_dict(), model_path)
    model_size = os.path.getsize(model_path)
    model_size = round(model_size / (1024 * 1024))
    return model_size

In [146]:
YOLO_MODEL_SIZE = get_model_size(model=model, model_path='yolo.pth')
FASTER_RCNN_MODEL_SIZE = get_model_size(model=model_02, model_path='fasterrcnn.pth')
SSD_MODEL_SIZE = get_model_size(model=model_03, model_path='ssd.pth')

In [147]:
YOLO_MODEL_SIZE, FASTER_RCNN_MODEL_SIZE, SSD_MODEL_SIZE

(12, 167, 136)

### Speed: FPS

* **YOLOv8:** 4 FPS
* **Faster RCNN:** 0.1 FPS
* **SSD:** 2 FPS

I am using CPU, so it is much slower than GPU. Your video processing speed might be faster than mine.

In [148]:
YOLO_FPS, FASTER_RCNN_FPS, SSD_FPS

(2.9513940976633943, 0.0, 0.0)

### Total Detected Cars

* **YOLOv8:** 4 cars
* **Faster RCNN:** 11 cars
* **SSD:** 1 car

In [149]:
number_of_cars_yolo = len(predictions_yolo[0].boxes)
number_of_cars_faster_rcnn = len(car_boxes_f)
number_of_cars_ssd = len(car_boxes_s)

number_of_cars_yolo, number_of_cars_faster_rcnn, number_of_cars_ssd

(4, 14, 0)

## Speed Estimation with YOLOv8

### My Speed Estimation Function

In [154]:
output = cv2.VideoWriter("yolov8-speed-estimation.mp4", cv2.VideoWriter_fourcc(*'DIVX'), 30, (1920, 1080))
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("Error opening video stream or file")
    exit()

num_frames = 0
start_time = time.time()

track_history = defaultdict(list)

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    num_frames += 1
    previous_frame_time = time.time()

    # YOLOv8 object tracking
    results = model.track(frame, persist=True, classes=2, show_labels=False, show_conf=False,
                            tracker="bytetrack.yaml")

    # Get the boxes and track IDs
    boxes = results[0].boxes.xywh.cpu()
    track_ids = results[0].boxes.id.cpu().tolist()

    # Visualize the results on the frame
    annotated_frame = results[0].plot()

    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        track = track_history[track_id]
        track.append((float(x), float(y)))  # x, y center point

    end_time = time.time()
    elapsed_time = previous_frame_time - start_time
    FPS = num_frames / elapsed_time
    time_interval = 1 / FPS 
   
    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        if len(track_history[track_id]) > 1:
            calculate_speed(current_center=track_history[track_id][-1],
                            previous_center=track_history[track_id][-2],
                            scaling_factor=SCALING_FACTOR,
                            time_interval=time_interval,
                            frame=annotated_frame,
                            points=(int(x-(w/2)), int(y-(h/2) -10)))

        # Draw the tracking lines
        points = np.hstack(track_history[track_id]).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(annotated_frame, [points], isClosed=False,
                        color=(0, 0, 0), thickness=5)

    cv2.imshow("YOLOv8 Tracking", annotated_frame)
    output.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
output.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 150.8ms
Speed: 25.3ms preprocess, 150.8ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 178.5ms
Speed: 5.0ms preprocess, 178.5ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 108.7ms
Speed: 2.4ms preprocess, 108.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 106.1ms
Speed: 3.0ms preprocess, 106.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 106.3ms
Speed: 4.0ms preprocess, 106.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 106.9ms
Speed: 2.3ms preprocess, 106.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 131.2ms
Speed: 2.0ms preprocess, 131.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 103.0ms
Speed: 2.0ms preprocess, 103.0ms inference, 1.0ms postprocess per image at shape (1, 3, 3

### YOLOv8 pre-built Speed Estimation function

In [152]:
model = YOLO("yolov8n.pt")
names = model.model.names

cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("pre_built_yolo_speed_estimation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

line_pts = [(0, h//2+80), (w, h//2+80)]

speed_obj = speed_estimation.SpeedEstimator()
speed_obj.set_args(
    reg_pts=line_pts,
    names=names,
    view_img=True,
)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(frame, persist=True, show=False)

    frame = speed_obj.estimate_speed(frame, tracks)
    video_writer.write(frame)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 1 traffic light, 118.6ms
Speed: 3.1ms preprocess, 118.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 traffic light, 148.7ms
Speed: 7.0ms preprocess, 148.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 traffic light, 110.3ms
Speed: 3.3ms preprocess, 110.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 traffic light, 115.7ms
Speed: 4.0ms preprocess, 115.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 traffic light, 105.7ms
Speed: 5.0ms preprocess, 105.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 traffic light, 102.6ms
Speed: 3.1ms preprocess, 102.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 traffic light, 97.6ms
Speed: 2.2ms preprocess, 97.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

